# Customer Segmentation with RFM

Business Problem-Bir e ticaret sirketi musterilerini segmentlere ayirip pazarlama stratejileri gelistirmek istiyor.

In [2]:
import pandas as pd
import datetime as dt

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)
pd.set_option("display.expand_frame_repr", False)

df_ = pd.read_excel("/Users/betulyilmaz/Desktop/Miuul/CRM Analytics/Datasets/online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [3]:
def check_df(dataframe):
    print('------------ Shape ------------')
    print(dataframe.shape)
    print('------------ Types ------------')
    print(dataframe.dtypes)
    print('------------ Describe ------------')
    print(dataframe.describe().T)
    print('------------ NA ------------')
    print(dataframe.isnull().sum())

check_df(df)

------------ Shape ------------
(525461, 8)
------------ Types ------------
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
------------ Describe ------------
                 count                           mean                  min                  25%                  50%                  75%                  max      std
Quantity    525461.000                         10.338            -9600.000                1.000                3.000               10.000            19152.000  107.424
InvoiceDate     525461  2010-06-28 11:37:36.845017856  2009-12-01 07:45:00  2010-03-21 12:20:00  2010-07-06 09:51:00  2010-10-15 12:45:00  2010-12-09 20:01:00      NaN
Price       525461.000                          4.689           -53594.360                1.250                2.100               

In [5]:
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)
df = df[~df["Invoice"].astype("str").str.contains("C", na=False)]

In [11]:
check_df(df)

------------ Shape ------------
(407695, 8)
------------ Types ------------
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
------------ Describe ------------
                 count                           mean                  min                  25%                  50%                  75%                  max      std
Quantity    407695.000                         13.587                1.000                2.000                5.000               12.000            19152.000   96.842
InvoiceDate     407695  2010-07-01 10:10:10.782177792  2009-12-01 07:45:00  2010-03-26 14:01:00  2010-07-09 15:46:00  2010-10-14 17:09:00  2010-12-09 20:01:00      NaN
Price       407695.000                          3.294                0.000                1.250                1.950               

In [12]:
# Unique urun sayisi
df["Description"].nunique()

4444

In [13]:
df["Description"].value_counts().head()

Description
WHITE HANGING HEART T-LIGHT HOLDER    3153
REGENCY CAKESTAND 3 TIER              1706
STRAWBERRY CERAMIC TRINKET BOX        1407
ASSORTED COLOUR BIRD ORNAMENT         1369
HOME BUILDING BLOCK WORD              1219
Name: count, dtype: int64

In [18]:
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,56915
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54754
BROCADE RING PURSE,48166
PACK OF 72 RETRO SPOT CAKE CASES,45156
ASSORTED COLOUR BIRD ORNAMENT,44551


In [22]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
df.groupby("Invoice").agg({"TotalPrice" : "sum"}).head()

,TotalPrice
Invoice,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240


Calculating RFM Metrics

Recency-when was the last time the customer made a purchase
Frequency-how often does the customer spend money
Monetary-how much did the customer spend

In [25]:
df["InvoiceDate"].max() # 2010-12-09
today_date = dt.datetime(2010, 12, 11)

In [27]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda x: (today_date-x.max()).days,
                                     "Invoice": lambda x: x.nunique(),
                                     "TotalPrice": lambda x: x.sum()})

rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [28]:
rfm.columns = ['recency', 'frequency', 'monetary']
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.802,349164.350


In [29]:
rfm = rfm[rfm["monetary"] > 0]


,count,mean,std,min,25%,50%,75%,max
recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


Calculating RFM Scores

In [32]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])

rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5]) # ayni araliklara farkli degerler gelecekken ayni gelme durumundan dolayi rank kullandik

rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])

In [33]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
12346.000,165,11,372.860,2,5,2
12347.000,3,2,1323.320,5,2,4
12348.000,74,1,222.160,2,1,1
12349.000,43,3,2671.140,3,3,5
12351.000,11,1,300.930,5,1,2


In [38]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

In [39]:
rfm[rfm["RFM_SCORE"] == "55"].head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12415.000,11,7,19543.840,5,5,5,55
12431.000,9,13,4370.520,5,5,5,55
12471.000,10,49,20139.740,5,5,5,55
12472.000,5,13,11308.480,5,5,5,55
12474.000,14,13,5048.660,5,5,5,55


In [37]:
rfm[rfm["RFM_SCORE"] == "11"].head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12355.000,203,1,488.210,1,1,2,11
12362.000,374,1,130.000,1,1,1,11
12366.000,269,1,500.240,1,1,2,11
12368.000,264,1,917.700,1,1,3,11
12378.000,198,1,1407.700,1,1,4,11


Creating & Analysing RFM Segments

In [41]:
#regex
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm["segment"] = rfm["RFM_SCORE"].replace(seg_map, regex = True)


rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])


recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       53.819   343     1.201   343  441.320   343
at_Risk             152.159   611     3.074   611 1188.878   611
cant_loose          124.117    77     9.117    77 4099.450    77
champions             7.119   663    12.554   663 6852.264   663
hibernating         213.886  1015     1.126  1015  403.978  1015
loyal_customers      36.287   742     6.830   742 2746.067   742
need_attention       53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loyalists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

In [42]:
# new_customers id bilgileri
rfm[rfm["segment"] == "new_customers"].index

Index([12346.0, 12380.0, 12482.0, 12510.0, 12891.0, 12932.0, 13044.0, 13313.0,
       13680.0, 13782.0, 13799.0, 13856.0, 14025.0, 14063.0, 14160.0, 14221.0,
       14548.0, 14607.0, 14685.0, 14745.0, 15003.0, 15013.0, 15015.0, 15125.0,
       15141.0, 15222.0, 15306.0, 15321.0, 15359.0, 15369.0, 15372.0, 15443.0,
       15538.0, 15607.0, 15633.0, 15722.0, 15751.0, 15754.0, 15768.0, 15911.0,
       15912.0, 16027.0, 16032.0, 16158.0, 16177.0, 16197.0, 16335.0, 16467.0,
       16631.0, 16742.0, 16743.0, 16875.0, 16986.0, 17021.0, 17032.0, 17092.0,
       17113.0, 17157.0, 17188.0, 17230.0, 17268.0, 17426.0, 17448.0, 17454.0,
       17512.0, 17578.0, 17602.0, 17651.0, 17940.0, 17969.0, 17988.0, 18009.0,
       18051.0, 18064.0, 18094.0, 18251.0, 18258.0],
      dtype='float64', name='Customer ID')

In [ ]:
new_df = pd.DataFrame()
new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index

new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)

new_df.to_csv("new_customers.csv")

rfm.to_csv("rfm.csv")